<h3>Kaggle Challenge</h3>
<h1>Expedia Hotel Recommendations</h1>
<hr style="height:2px;border:none;color:#333;background-color:#333;"/>
<b>Part II - Data Processing</b>

### Imports

In [1]:
#Python Libraries
import numpy as np
import pandas as pd
import time
import datetime

#Graphs libraries
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

#Modeling and Pre-processing libraries
import ml_metrics as metrics
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import LabelEncoder


##Global Parameters
pd.set_option('display.max_columns', 500)
sns.set(style="whitegrid")
matplotlib.rcParams['figure.figsize'] = (10.0, 7.0)
%matplotlib inline

## 1. Load the dataset

In [5]:
%%time
ROWS = None
# ROWS = 1000000

expediaDF = pd.read_csv('data/train.csv.gz', 
                        nrows=ROWS,
                        compression='gzip',
                        error_bad_lines=False)

CPU times: user 4min 11s, sys: 14.8 s, total: 4min 26s
Wall time: 4min 15s


In [6]:
print("Shape:", expediaDF.shape)
expediaDF.head()

Shape: (37670293, 24)


,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,9,2014-08-27,2014-08-31,2,0,1,8250,1,0,3,2,50,628,1
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.2641,12,0,1,9,2014-08-29,2014-09-02,2,0,1,8250,1,1,1,2,50,628,1
2,2014-08-11 08:24:33,2,3,66,348,48862,2234.2641,12,0,0,9,2014-08-29,2014-09-02,2,0,1,8250,1,0,1,2,50,628,1
3,2014-08-09 18:05:16,2,3,66,442,35390,913.1932,93,0,0,3,2014-11-23,2014-11-28,2,0,1,14984,1,0,1,2,50,1457,80
4,2014-08-09 18:08:18,2,3,66,442,35390,913.6259,93,0,0,3,2014-11-23,2014-11-28,2,0,1,14984,1,0,1,2,50,1457,21


In [7]:
print(datetime.datetime.now())
testingDF = pd.read_csv('data/test.csv.gz', 
#                         nrows=100000,
                        compression='gzip',
                        error_bad_lines=False)
print(datetime.datetime.now())

2019-05-09 10:40:40.384450
2019-05-09 10:40:53.250320


In [5]:
print("Shape:", testingDF.shape)
testingDF.head()

Shape: (2528243, 22)


,id,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,hotel_continent,hotel_country,hotel_market
0,0,2015-09-03 17:09:54,2,3,66,174,37449,5539.0567,1,1,0,3,2016-05-19,2016-05-23,2,0,1,12243,6,6,204,27
1,1,2015-09-24 17:38:35,2,3,66,174,37449,5873.2923,1,1,0,10,2016-05-12,2016-05-15,2,0,1,14474,7,6,204,1540
2,2,2015-06-07 15:53:02,2,3,66,142,17440,3975.9776,20,0,0,1,2015-07-26,2015-07-27,4,0,1,11353,1,2,50,699
3,3,2015-09-14 14:49:10,2,3,66,258,34156,1508.5975,28,0,1,10,2015-09-14,2015-09-16,2,0,1,8250,1,2,50,628
4,4,2015-07-17 09:32:04,2,3,66,467,36345,66.7913,50,0,0,0,2015-07-22,2015-07-23,2,0,1,11812,1,2,50,538


## 2. Data Cleaning

### Handle missing Values

#### - Training Dataset

In [8]:
missingValues = expediaDF.isnull().sum().sort_values(ascending=False)
missingValues = missingValues.to_frame().reset_index()
missingValues.columns = ["feature", "count"]
missingValues["percentage"] = missingValues["count"] / expediaDF.shape[0]
print("Missing Values: ")
missingValues[missingValues["count"] > 0]

Missing Values: 


,feature,count,percentage
0,orig_destination_distance,13525001,0.359036
1,srch_co,47084,0.001250
2,srch_ci,47083,0.001250


- Removing checkin and checkout null rows

In [9]:
%%time
countBefore = expediaDF.shape[0]
# newExpediaDF = expediaDF.dropna(subset=['srch_co', 'srch_ci','orig_destination_distance'])
newExpediaDF = expediaDF.dropna(subset=['srch_co', 'srch_ci'])
countAfter = newExpediaDF.shape[0]
deletedRows = countBefore - countAfter
print("Rows deleted: ", deletedRows, "- % of the dataset: ", "{:.2%}".format(deletedRows / countBefore))

Rows deleted:  47088 - % of the dataset:  0.13%
CPU times: user 18.3 s, sys: 2.36 s, total: 20.6 s
Wall time: 16.3 s


In [10]:
newExpediaDF['orig_destination_distance'] = \
    newExpediaDF['orig_destination_distance'].fillna(newExpediaDF['orig_destination_distance'].mean())

/home/sbarboza/yes/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
newExpediaDF.shape

(37623205, 24)

In [12]:
del expediaDF

#### - Testing Dataset

In [11]:
missingValues = testingDF.isnull().sum().sort_values(ascending=False)
missingValues = missingValues.to_frame().reset_index()
missingValues.columns = ["feature", "count"]
missingValues["percentage"] = missingValues["count"] / testingDF.shape[0]
print("Missing Values: ")
missingValues[missingValues["count"] > 0]

Missing Values: 


,feature,count,percentage
0,orig_destination_distance,847461,0.335198
1,srch_ci,21,0.000008
2,srch_co,17,0.000007


- Handling missing values for checkin and checkout

In [ ]:
# testingDF = testingDF.fillna(0)

In [12]:
#Transform the datatype of checkin and checkout in date
newExpediaDF['srch_ci'] = pd.to_datetime(newExpediaDF['srch_ci'], errors='coerce')
newExpediaDF['srch_co'] = pd.to_datetime(newExpediaDF['srch_co'], errors='coerce')

testingDF['srch_ci'] = pd.to_datetime(testingDF['srch_ci'], errors='coerce')
testingDF['srch_co'] = pd.to_datetime(testingDF['srch_co'], errors='coerce')


In [13]:
#Handle missing values of checkin and checkout

#fill with one day difference between checkin/out
testingDF['srch_ci'] = testingDF['srch_ci'].fillna(testingDF['srch_co'] - datetime.timedelta(1))
testingDF['srch_co'] = testingDF['srch_co'].fillna(testingDF['srch_ci'] + datetime.timedelta(1))

#fill the left with the mode
testingDF['srch_ci'] = testingDF['srch_ci'].fillna(testingDF['srch_ci'].mode()[0])
testingDF['srch_co'] = testingDF['srch_co'].fillna(testingDF['srch_co'].mode()[0])

- Handling missing values for orig_destination_distance

In [105]:
mean = newExpediaDF.groupby(['user_location_region', 'srch_destination_id'])['orig_destination_distance']\
        .mean().to_frame().reset_index()

In [77]:
missingDF = testingDF[testingDF.orig_destination_distance.isnull()]
df = missingDF.groupby(['user_location_region', 'srch_destination_id']).size().to_frame().reset_index()
df = df.drop([0], axis=1)
df.head()

,user_location_region,srch_destination_id
0,0,19
1,0,21
2,0,24
3,0,27
4,0,40


In [14]:
newTestingDF = testingDF.copy()

In [ ]:
%%time
regions = df.user_location_region.unique()
newTestingDF = testingDF.copy()
count = 0
res = []

for region in regions:
    destinations = df[df['user_location_region'] == region].srch_destination_id.unique()
    
    for dest in destinations:
        origDestMean = mean[ \
        (mean.user_location_region == region) & \
        (mean.srch_destination_id == dest) \
        ]['orig_destination_distance']
        
        if(origDestMean.size > 0):
            newTestingDF.loc[
                (newTestingDF['orig_destination_distance'].isnull()) &
                (newTestingDF['user_location_region'] == region) &
                (newTestingDF['srch_destination_id'] == dest),
                'orig_destination_distance'
            ] = origDestMean.values[0]
            res.append(origDestMean)
        
        count += 1
        if(count % 100 == 0):
            print(count)
        

In [15]:
# where could not find any value for same user_location_region and  srch_destination_id on training set:
# set the mean of orig_destination_distance in testing set.
newTestingDF['orig_destination_distance'] = \
    newTestingDF['orig_destination_distance'].fillna(newTestingDF['orig_destination_distance'].mean())

In [16]:
testingDF = newTestingDF
del newTestingDF

In [17]:
#Checking if all the missing values were eliminated
assert testingDF.isnull().sum().sum() == 0
assert newExpediaDF.isnull().sum().sum() == 0

- Include the is_booking column (all the testing data are booking)

In [18]:
testingDF['is_booking'] = 1

### Sampling the data

The strategy here is to build a model on the last 6 months of 2014, from '2014-07-01' onwards in order to mirror the situation in the test set.

In [ ]:
# %%time
# newExpediaDF['date_time'] = newExpediaDF['date_time'].astype('datetime64[ns]')
# newExpediaDF = newExpediaDF[newExpediaDF['date_time'] > datetime.date(2014,7,1)]
# print("Shape: ",newExpediaDF.shape)

In [114]:
# newExpediaDF = newExpediaDF[newExpediaDF['is_booking'] == 1]

In [ ]:
##Sampling the data for test models
#Comment if you want to model the whole data
N = 1000000
trainingDF = trainingDF.sample(n=N)

In [14]:
train_cleaned = newExpediaDF

### Handle the outliers

In [16]:
%%time
#removing outliers
countBefore = newExpediaDF.shape[0]
num_train = newExpediaDF.select_dtypes(include=["number"])
cat_train = newExpediaDF.select_dtypes(exclude=["number"])
idx = np.all(stats.zscore(num_train) < 7, axis=1)
countAfter = np.sum(idx)
deletedRows = countBefore - countAfter
train_cleaned = pd.concat([num_train.loc[idx], cat_train.loc[idx]], axis=1)
print("Rows deleted: ", deletedRows, "- % of the dataset: ", "{:.2%}".format(deletedRows / countBefore))

Rows deleted:  221441 - % of the dataset:  0.59%
CPU times: user 39.1 s, sys: 32.1 s, total: 1min 11s
Wall time: 1min 6s


In [17]:
%%time
del num_train
del cat_train
del idx
del newExpediaDF
del countBefore
del countAfter

CPU times: user 680 ms, sys: 128 ms, total: 807 ms
Wall time: 800 ms


### Handle categorical values

In [18]:
%%time
def encode_label(df):
    X_cat = df.copy()
    X_cat = df.select_dtypes(include=['object'])
    X_enc = X_cat.copy()
    X_enc = X_enc.apply(LabelEncoder().fit_transform)
    mergedata = df.drop(X_cat.columns, axis=1)

    return pd.concat([mergedata,X_enc], axis=1).astype(float)

CPU times: user 116 µs, sys: 15 µs, total: 131 µs
Wall time: 26.9 µs


In [19]:
train_cleaned["date_time"] = train_cleaned["date_time"].astype(str)
train_cleaned["srch_ci"] = train_cleaned["srch_ci"].astype(str)
train_cleaned["srch_co"] = train_cleaned["srch_co"].astype(str)

testingDF["date_time"] = testingDF["date_time"].astype(str)
testingDF["srch_ci"] = testingDF["srch_ci"].astype(str)
testingDF["srch_co"] = testingDF["srch_co"].astype(str)

In [20]:
%%time
train_cleaned = encode_label(train_cleaned)
# testingDF = encode_label(testingDF)

CPU times: user 8min 10s, sys: 1min 30s, total: 9min 41s
Wall time: 8min


In [21]:
train_cleaned.shape

(37401764, 24)

In [22]:
trainingDF = train_cleaned
del train_cleaned

In [23]:
%%time
trainingDF.to_csv("all_dataset_encoded_zscore3.csv")
# testingDF.to_csv("test_cleaned.csv")

CPU times: user 34min 57s, sys: 35 s, total: 35min 32s
Wall time: 35min 32s


## 3. Data Processing

### Load the cleaned data
Run this code if you have the cleaned data saved

In [25]:
%%time
# Uncomment if you want to read the train already cleaned
# trainingDF = pd.read_csv('train_cleaned.csv')
# trainingDF = trainingDF.drop(columns='Unnamed: 0')

testingDF = pd.read_csv('test_cleaned_and_encoded.csv')
testingDF = testingDF.drop(columns='Unnamed: 0')

CPU times: user 16.8 s, sys: 324 ms, total: 17.1 s
Wall time: 14.1 s


In [26]:
#Check if the testing set has the correct number of rows
assert testingDF.shape[0] == 2528243, "Wrong number of testing set"

In [27]:
print("Training set - Percentage of the whole dataset: {:.0f}%".format(trainingDF.shape[0]*100/37670293))

Training set - Percentage of the whole dataset: 99%


### Split the data

In [28]:
%%time
X = trainingDF.drop('hotel_cluster', axis=1)
y = trainingDF["hotel_cluster"]

CPU times: user 2.55 s, sys: 4.01 s, total: 6.56 s
Wall time: 5.97 s


In [29]:
%%time
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

CPU times: user 34.5 s, sys: 4.91 s, total: 39.4 s
Wall time: 39.2 s


In [30]:
print("Training with {:.2f}% of the expedia dataset".format((X_train.shape[0] / 37670293)*100))

Training with 89.36% of the expedia dataset


### Scaling the data

In [31]:
%%time
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_submission = scaler.transform(testingDF)

CPU times: user 22.8 s, sys: 10.2 s, total: 33 s
Wall time: 9.98 s


### Modeling

In [32]:
%time
def predictAndEvaluate(model):
    print(datetime.datetime.now())

    m = model().fit(X_train, y_train)
    print('Accuracy on training set: {:.2f}'.format(m.score(X_train, y_train)))
    print('Accuracy on test set: {:.2f}'.format(m.score(X_test, y_test)))
    print(datetime.datetime.now())
    y_test_predicted_probability = m.predict_proba(X_test)

    prob = y_test_predicted_probability.argsort()
    predictions = []
    for p in prob:
        predictions.append(list(reversed(p[-5:])))
        
    targ = [[l] for l in y_test]
    score = metrics.mapk(targ, predictions, k=5)
    print('Accuracy of the predictions (MAP@5): {:.2f}'.format(score))
    return m

CPU times: user 10 µs, sys: 1 µs, total: 11 µs
Wall time: 19.8 µs


- <b>Decision Tree</b>

In [33]:
%%time
from sklearn.tree import DecisionTreeClassifier
dtc = predictAndEvaluate(DecisionTreeClassifier)

2019-05-09 11:47:34.339069
Accuracy on training set: 1.00
Accuracy on test set: 0.33
2019-05-09 12:56:40.915552
Accuracy of the predictions (MAP@5): 0.34
CPU times: user 1h 8min 52s, sys: 4min 37s, total: 1h 13min 29s
Wall time: 1h 13min 13s


In [42]:
%%time
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = predictAndEvaluate(LinearDiscriminantAnalysis)

2019-05-09 13:27:31.213174
Accuracy on training set: 0.06
Accuracy on test set: 0.06
2019-05-09 13:32:46.279033
Accuracy of the predictions (MAP@5): 0.11
CPU times: user 9min 47s, sys: 1min 16s, total: 11min 4s
Wall time: 7min 42s


In [43]:
best_model = lda

### Generating result file

In [44]:
%%time
print(datetime.datetime.now())
predicted_probability = best_model.predict_proba(X_submission)

prob = predicted_probability.argsort()

result = []
for p in prob:
    predictions = list(reversed(p[-5:]))
    strPredictions = " ".join(str(v) for v in predictions)
    result.append(strPredictions)
    
resultDF = pd.DataFrame(result, columns=['hotel_cluster'])
resultDF = pd.concat([testingDF.id, resultDF.hotel_cluster], axis=1)

print(datetime.datetime.now())
resultDF.hotel_cluster.to_csv('predicted_with_pandas_lda_24.csv',header=True, index_label='id')

2019-05-09 13:35:13.334911
2019-05-09 13:38:05.784122
CPU times: user 3min 1s, sys: 4.38 s, total: 3min 6s
Wall time: 3min
